# Reading Data

Earlier, we mentioned that most of our notebooks, and even some of our individual cells, follow a fairly standard pattern:

1. **Read Data**
2. Clean Data
3. Filter Data
4. Process Data
5. Output Data

This article will be about reading data.

The first things to think about when reading data into a pandas dataframe are as follows:

1. Is the data 2D columnar data?  Pandas dataframes store data much as an Excel spreadsheet. If you can imagine storing the data as a spreadsheet, then pandas can probably read it into a dataframe.  On the other hand, deeply nested tree structures like xml and json may not be something you can read.
2. Will the data fit in memory?  Our development machines are pretty big, but there are limits to how much data they can read in. Keep this in mind before you open up a 2TB csv file.  Check that you have the needed resources before starting.

While it is possible to read in data by your self and convert it to a dataframe, it is generally better to use the built in pandas read methods.


Here are some of the most common ways to read data into a Pandas DataFrame:

### `read_csv()`

This function allows you to read data from a CSV file into a DataFrame. It's one of the most used functions due to the prevalence of CSV files. You can specify parameters like separator, columns to load, data types, and much more. Here's an example of its usage:

```python
import pandas as pd 
df = pd.read_csv('filename.csv') 
```

### `read_excel()` 

If your data is stored in an Excel file, this function is what you need. You can also specify the sheet name to be read in the function. An example would be:

```python
df = pd.read_excel('filename.xlsx', sheet_name='Sheet1') 
```

### `read_sql_query()` 

This function is used when you need to load the result of an SQL query into a DataFrame. You need to have an open connection to a database. An example would be:

```python
import sqlite3 connection = sqlite3.connect("database.db") df = pd.read_sql_query("SELECT * FROM table", connection) 
```

### `read_json()`
 
JSON is a popular data format, especially when dealing with APIs. This function allows you to load JSON data into a DataFrame:

```python
df = pd.read_json('filename.json') 
```
Note that json is not inherently tabular, so you may only be able to read in certain kinds of json structures, and you may have to tell the `read_json` function which one of these structures its looking at to help it read your json.

### `read_html()`

With this function, you can extract tables directly from an HTML page. This can be particular useful for web scraping tasks such as reading a list of ISO country codes from a wikipedia article.

```python
df_list = pd.read_html('http://website.com/my_table') 
```

Do note that read_html() returns a list of DataFrames, you need to select the correct one from the list.

Pandas also supports many other types of data sources like Parquet, SAS, Stata, etc. The function naming convention is straightforward, so you can easily determine which function to use.

Just like the other `read_*` methods, using `read_parquet()` to load Parquet files into a Pandas DataFrame is straightforward.

Parquet is a columnar format that is supported by many other data processing systems, and for which Pandas provides support through Apache's pyarrow library.

First, you will need to make sure `pyarrow` is installed, as pandas requires this library to read/write Parquet files. You can install it using pip:

```bash
pip install pyarrow
```

Once you have `pyarrow` or `fastparquet` installed, here's an example of how to use the `read_parquet` function:

```python
import pandas as pd

df = pd.read_parquet('filename.parquet')
```

The 'filename.parquet' is the path to your Parquet file. If your Parquet file is in the same directory as your Python script or Jupyter notebook, then you only need to provide the name of the file. If the file is located elsewhere, you'll need to provide the full path to the file.

Remember to replace 'filename.parquet' with the path to your actual Parquet file. After running these lines, df will hold the data from the Parquet file and you can use the DataFrame as usual.

## Pandas DataFrame Reading Considerations

**Data Type Detection:** When reading in a file, Pandas tries to infer the data types of columns based on the contents, which can sometimes lead to unexpected results. For example, a column containing numeric codes with leading zeros may be interpreted as integers, thus stripping the zeros. You may want to specify dtype=str for such columns.  Similarly, if the first several thousand rows are integers and the rest are floats, the type detection code may get confused and think the column is an integer and not a float.  Again, it is possible to manually specify the types if needed.  

In general, formats like CSV and Excel can be problematic as their data contains basically just text. There is no data in the file to tell anyone reading the data what the column actually contains.  When reading in CSV, you will have more cleanup code to do to force the data to be interpreted correctly.  While there is often no choice in reading from a csv file, if possible, please save your data in a format that will properly capture the type of data so that the next person that reads it wont have to work out what the column types are manually. Parquet files are quite good at providing small, fast, and type safe data storage.

**Memory Usage:** Large files can consume a lot of memory. If you're reading in a large file, consider reading the file in chunks, or only reading in certain columns.

**Missing Data:** Pandas represents missing data as NaN which can sometimes cause unexpected results in your calculations or data manipulations. Always check for missing data after loading your dataframe.

**Indexing:** By default, Pandas will create a new index for the DataFrame. If you want to use one of the columns as the index, you need to explicitly specify it.

**Parse Dates:** If your dataset contains dates, it's important to ensure that these are correctly parsed as datetime objects, not strings. You might need to provide parse_dates parameter when reading the file, or provide parsing operations after the fact (for instance if the data you read expresses times as the number of fractional seconds past the epoch).

**Encoding issues:** Not all CSV files are created with the same encoding. If you try to read a file with an unusual encoding, you may encounter a UnicodeDecodeError. In such cases, you’ll need to specify the correct encoding.

**Inconsistent Data:** Real-world data can be messy. There could be issues with inconsistent entries, outliers, unusual delimiters, or mis-structured records.

This can be a considerable task when reading things, such as excel files, that were written by humans
for humans.  Humans may decide to stick random comments into cells.  Another human reading that may understand the context and not think it was the name of a data center, but pandas wont be so forgiving.  Similarly, humans may color code records such that a person will know if the data is correct, for instance, but the `read_excel` file will not understand any of that coloring (or font choices) and you will lose that information.  Humans also often only fill in the first row of a series of similar values, and the reading human may know that this applies to every row from here to the next one where we specified a new value.  Pandas will not know this by default.

So, when reading human created csv and excel files, be very careful to make sure you know what it meant, and how to interpret it.  If possible, push back to the source and have them make a very simple, regular, well organized data set. 


**Escape Characters:** In CSV files some fields can contain delimiter or newline characters. To correctly read these files, you need to specify escapechar, quotechar or other parameters.

It's a good practice to always start by reading a small chunk of the file or the first few rows (pd.read_csv(file, nrows=5)) to check that everything is being interpreted correctly.